In [1]:
import os
directory = os.getcwd()  
directory = directory.split('src')
path = directory[0]
%cd $path
%pwd

c:\Users\JENMOSQU\Documents\PERSONAL\cod\Probabilistic-Electricity-Demand-


'c:\\Users\\JENMOSQU\\Documents\\PERSONAL\\cod\\Probabilistic-Electricity-Demand-'

In [2]:
# Libraries..

import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
from itertools import product
from tqdm import tqdm_notebook
import pandas as pd
import copy

# Plotly

import plotly.graph_objects as go
import plotly.express as px
import plotly.graph_objs as go
import plotly.offline as pyo
import plotly.graph_objects as go



# sklearn
from sklearn.compose import make_column_transformer

from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error,mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit

# Keras

from keras.models import Sequential
from keras.backend import clear_session
from keras.callbacks import EarlyStopping
from keras_tuner.tuners import RandomSearch
from keras.layers import SimpleRNN, LSTM, GRU, Dropout, Dense, Bidirectional

# tensorflow
from tensorflow.keras.models import load_model

#skforecast
from skforecast.utils import save_forecaster
from skforecast.utils import load_forecaster

# tensorflow
from tensorflow.keras.models import load_model


from scipy.stats import randint
from mapie.metrics import (regression_coverage_score,regression_mean_width_score)
from mapie.subsample import BlockBootstrap
from mapie.regression import MapieTimeSeriesRegressor


# Modules from repo
from src.Utilities import utilities as tool



# Configuración warnings
# ==============================================================================
import warnings
warnings.filterwarnings('ignore')
# Configuración options.display
# ==============================================================================
pd.set_option('display.max_rows', 5000)
pd.options.display.max_rows = None
pd.options.display.max_columns = 5000
pd.options.display.max_colwidth = 5000
pd.options.display.max_dir_items = 5000
pd.options.display.width = 5000
pd.options.display.float_format = '{:.3f}'.format


Using TensorFlow backend


In [3]:
def plot_forecast(y_train, y_test, y_preds, y_pis, coverages, widths, plot_coverage=True):
    for i, w in enumerate(["without", "with"]):
        fig = go.Figure()

        # Training data plot with correct date range
        fig.add_trace(go.Scatter(
            x=y_train.index,
            y=y_train,
            mode='lines',
            name='Training data',
            line=dict(color='#bc8f8f', width=2),
        ))
        # Prediction intervals plot with updated color
        fig.add_trace(go.Scatter(
            x=list(y_test.index) + list(y_test.index)[::-1],
            y=list(y_pis[i][:, 0, 0]) + list(y_pis[i][:, 1, 0])[::-1],
            fill='toself',
            fillcolor='rgba(185, 203, 211, 0.8)',  # Cambio de color aquí
            line=dict(color='rgba(0,0,0,0)'),
            name='Prediction intervals',
        ))
        
        # Test data plot with updated color
        fig.add_trace(go.Scatter(
            x=y_test.index,
            y=y_test,
            mode='lines',
            name='Test data',
            line=dict(color='#b98170', width=2),  # Cambio de color aquí
        ))

        # Predictions plot with updated color
        fig.add_trace(go.Scatter(
            x=y_test.index,
            y=y_preds[i],
            mode='lines',
            name='Predictions',
            line=dict(color='#276981', width=2),
        ))


        title = f"EnbPI, {w} update of residuals. "
        if plot_coverage:
            title += f"Coverage: {coverages[i]:.3f} and Width: {widths[i]:.3f}"

        # Update the layout for each subplot
        fig.update_layout(
            title=title,
            xaxis_title='Date-daily',
            yaxis_title='Conformal Prediction Interval',
            showlegend=True,
        )

        fig.show()

In [4]:
#Import Data..............

df=pd.read_parquet('df.parquet')
columnas_indeseadas =['HR_CAL_MEDIA_D', 'VVAG_MEDIA_D'] #columna con correlación 0 con la demanda
columnas_indeseadas += [f'Demanda_SIN_{i}' for i in range(1, 29)]
# columnas_indeseadas += [f'Demanda_SIN_{i}' for i in range(7, 29, 7)]
print("columnas rezagadas que no son necesarias porque skforecast rezaga internamente",columnas_indeseadas)
df = df.loc[:, ~df.columns.isin(columnas_indeseadas)]
print(df.head(1))
print("--------------------------------------------------------------------------------------------------")
print(df.tail(1))

columnas rezagadas que no son necesarias porque skforecast rezaga internamente ['HR_CAL_MEDIA_D', 'VVAG_MEDIA_D', 'Demanda_SIN_1', 'Demanda_SIN_2', 'Demanda_SIN_3', 'Demanda_SIN_4', 'Demanda_SIN_5', 'Demanda_SIN_6', 'Demanda_SIN_7', 'Demanda_SIN_8', 'Demanda_SIN_9', 'Demanda_SIN_10', 'Demanda_SIN_11', 'Demanda_SIN_12', 'Demanda_SIN_13', 'Demanda_SIN_14', 'Demanda_SIN_15', 'Demanda_SIN_16', 'Demanda_SIN_17', 'Demanda_SIN_18', 'Demanda_SIN_19', 'Demanda_SIN_20', 'Demanda_SIN_21', 'Demanda_SIN_22', 'Demanda_SIN_23', 'Demanda_SIN_24', 'Demanda_SIN_25', 'Demanda_SIN_26', 'Demanda_SIN_27', 'Demanda_SIN_28']
       FECHA TIPO_DIA  Demanda_SIN  Covid  Cambio_nivel  Tiempo  precipitacion  Temperatura  HRA2_MN_D  HRA2_MX_D  THSM_MEDIA_D  THSM_MX_D  VV_10_MEDIA_D  VV_10_VECT_MEDIA_D  THSM_MN_D  PT_2_TT_D  PT_2_MX_D  PT_10_MX_D  PT_10_TT_D  TSSM_MEDIA_D  TSSM_MN_D  TSSM_MX_D  PTPG_TT_D  BSHG_TT_D  TPR_CAL
0 2017-12-01  VIERNES   193865.750      0             0       1          9.589       22.203  

In [5]:
#Correlación variables y filtrado........
#['THSM_MEDIA_D','PTPG_TT_D']
correlacion_con_demanda = df.corr()['Demanda_SIN']
nombres_variables = correlacion_con_demanda[(correlacion_con_demanda >= 0.2) | (correlacion_con_demanda <= -0.2)]
print(nombres_variables)
nombres_variables=nombres_variables.index.tolist()
nombres_variables +=['THSM_MEDIA_D'] #'PTPG_TT_D' #'VVAG_MEDIA_D'
nombres_variables.remove('Demanda_SIN')
df=df[['FECHA', 'TIPO_DIA', 'Demanda_SIN'] + nombres_variables]
print("columnas finales", df.columns)
df.head(2)

Demanda_SIN           1.000
Covid                -0.246
Cambio_nivel          0.352
Tiempo                0.595
VV_10_MEDIA_D         0.213
VV_10_VECT_MEDIA_D    0.210
PT_10_TT_D           -0.291
Name: Demanda_SIN, dtype: float64
columnas finales Index(['FECHA', 'TIPO_DIA', 'Demanda_SIN', 'Covid', 'Cambio_nivel', 'Tiempo', 'VV_10_MEDIA_D', 'VV_10_VECT_MEDIA_D', 'PT_10_TT_D', 'THSM_MEDIA_D'], dtype='object')


,FECHA,TIPO_DIA,Demanda_SIN,Covid,Cambio_nivel,Tiempo,VV_10_MEDIA_D,VV_10_VECT_MEDIA_D,PT_10_TT_D,THSM_MEDIA_D
0,2017-12-01,VIERNES,193865.750,0,0,1,1.703,0.935,6.232,19.113
1,2017-12-02,SABADO,184393.750,0,0,2,1.757,0.930,7.888,19.166


In [6]:
#Rolling mean.........
columns_rolling = ['VV_10_MEDIA_D', 'VV_10_VECT_MEDIA_D','PT_10_TT_D' ,'THSM_MEDIA_D']
window_size=60
df = tool.exploratory_tools.fill_null_values_with_rolling_mean(df, columns_rolling, window_size)

In [7]:
# df.dropna(inplace=True)

In [7]:
# Transformer one hot encoding and formatting-.......

print("--------------------------------------------------------------------------------------------------------------------------------------")
# ==============================================================================
one_hot_encoder = make_column_transformer(
                      #(StandardScaler(), ['Demanda_SIN']),
                      (   
                          OneHotEncoder(sparse_output=False, drop='if_binary'),
                          ['TIPO_DIA'],
                      ),
                      remainder="passthrough",
                      verbose_feature_names_out=False,
                  ).set_output(transform="pandas")

df = one_hot_encoder.fit_transform(df)

#-------------------------------------------------------------------------------------------------------------------------------------------
#Variable TIPO_DIA
#-------------------------------------------------------------------------------------------------------------------------------------------
columnas_tipo_dia = [col for col in df.columns if col.startswith('TIPO_DIA_')]
variables_exogenas = ['FECHA'] + nombres_variables + columnas_tipo_dia
variable_dependiente = ['Demanda_SIN']
df = df[variable_dependiente + variables_exogenas] 

#-------------------------------------------------------------------------------------------------------------------------------------------
# Codificación de Fecha...
#-------------------------------------------------------------------------------------------------------------------------------------------
df['FECHA'] = pd.to_datetime(df['FECHA'], format='%Y-%m-%d')
df = df.set_index('FECHA')
df = df.asfreq('D')

--------------------------------------------------------------------------------------------------------------------------------------


In [8]:
# Separación datos train-val-test................
#-------------------------------------------------------------------------------------------------------------------------------------------

datos = df[df.index >= '2017-12-01']
fin_train = '2023-09-23'

datos_train = datos[datos.index <= fin_train].copy()
datos_test = datos[datos.index > fin_train].copy()

print(f"Fechas train      : {datos_train.index.min()} --- {datos_train.index.max()}  (n={len(datos_train)})")
print(f"Fechas test       : {datos_test.index.min()} --- {datos_test.index.max()}  (n={len(datos_test)})")
print("-------------------------------------------------------------------------------------------------------------------------------------")



variables_exogenas.remove('FECHA')
print(variables_exogenas)

Fechas train      : 2017-12-01 00:00:00 --- 2023-09-23 00:00:00  (n=2123)
Fechas test       : 2023-09-24 00:00:00 --- 2023-09-30 00:00:00  (n=7)
-------------------------------------------------------------------------------------------------------------------------------------
['Covid', 'Cambio_nivel', 'Tiempo', 'VV_10_MEDIA_D', 'VV_10_VECT_MEDIA_D', 'PT_10_TT_D', 'THSM_MEDIA_D', 'TIPO_DIA_1--ENE', 'TIPO_DIA_1--MAY', 'TIPO_DIA_2--ENE', 'TIPO_DIA_20--JUL', 'TIPO_DIA_24--DIC', 'TIPO_DIA_25--DIC', 'TIPO_DIA_31--DIC', 'TIPO_DIA_7--AGO', 'TIPO_DIA_8--DIC', 'TIPO_DIA_DOALF', 'TIPO_DIA_DOALFENE', 'TIPO_DIA_DOMINGO', 'TIPO_DIA_DOVDIC', 'TIPO_DIA_DOVENE', 'TIPO_DIA_DSS', 'TIPO_DIA_JSS', 'TIPO_DIA_JUEVES', 'TIPO_DIA_JUVDIC', 'TIPO_DIA_JUVENE', 'TIPO_DIA_LF', 'TIPO_DIA_LFENE', 'TIPO_DIA_LUNES', 'TIPO_DIA_LUVDIC', 'TIPO_DIA_LUVENE', 'TIPO_DIA_MADLF', 'TIPO_DIA_MARTES', 'TIPO_DIA_MAVDIC', 'TIPO_DIA_MAVENE', 'TIPO_DIA_MIERCOLES', 'TIPO_DIA_MIVDIC', 'TIPO_DIA_MIVENE', 'TIPO_DIA_MSS', 'TIPO_DIA_SAALF

In [9]:
# Organizar Data train y Data Test.....
X_train = datos_train.loc[~np.any(datos_train[variables_exogenas].isnull(), axis=1), variables_exogenas]
y_train = datos_train.loc[X_train.index, "Demanda_SIN"]
X_test = datos_test.loc[:, variables_exogenas]
y_test = datos_test["Demanda_SIN"]

### Parameters

In [98]:
alpha = 0.01 #0.01 #represents the uncertainty of the confidence interval. Lower alpha produce larger (more conservative) prediction intervals. 
gap = 1
n_iter = 100
n_splits = 5
random_state = 123

### Load model

In [99]:
lasso = load_forecaster('Ml_models/forecaster_lasso.py', verbose=True)
lasso.regressor

ForecasterAutoreg 
Regressor: Lasso(alpha=1.821885343465165e-05, random_state=123) 
Lags: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21] 
Transformer for y: StandardScaler() 
Transformer for exog: None 
Window size: 21 
Weight function included: False 
Differentiation order: None 
Exogenous included: True 
Type of exogenous variable: <class 'pandas.core.frame.DataFrame'> 
Exogenous variables names: ['Covid', 'Cambio_nivel', 'Tiempo', 'VV_10_MEDIA_D', 'VV_10_VECT_MEDIA_D', 'PT_10_TT_D', 'THSM_MEDIA_D', 'TIPO_DIA_1--ENE', 'TIPO_DIA_1--MAY', 'TIPO_DIA_2--ENE', 'TIPO_DIA_20--JUL', 'TIPO_DIA_24--DIC', 'TIPO_DIA_25--DIC', 'TIPO_DIA_31--DIC', 'TIPO_DIA_7--AGO', 'TIPO_DIA_8--DIC', 'TIPO_DIA_DOALF', 'TIPO_DIA_DOALFENE', 'TIPO_DIA_DOMINGO', 'TIPO_DIA_DOVDIC', 'TIPO_DIA_DOVENE', 'TIPO_DIA_DSS', 'TIPO_DIA_JSS', 'TIPO_DIA_JUEVES', 'TIPO_DIA_JUVDIC', 'TIPO_DIA_JUVENE', 'TIPO_DIA_LF', 'TIPO_DIA_LFENE', 'TIPO_DIA_LUNES', 'TIPO_DIA_LUVDIC', 'TIPO_DIA_LUVENE', 'TIPO_DIA_MADLF', 'TIPO_D

Lasso(alpha=1.821885343465165e-05, random_state=123)

In [100]:
lasso.regressor.get_params(alpha)

{'alpha': 1.821885343465165e-05,
 'copy_X': True,
 'fit_intercept': True,
 'max_iter': 1000,
 'positive': False,
 'precompute': False,
 'random_state': 123,
 'selection': 'cyclic',
 'tol': 0.0001,
 'warm_start': False}

In [101]:
# CV parameter search.......................
from sklearn.linear_model import Ridge, Lasso

model_params_fit_not_done = False
if model_params_fit_not_done:
    tscv = TimeSeriesSplit(n_splits=n_splits)
    rf_model = Lasso(random_state=random_state)
    rf_params = {'alpha': np.logspace(-10, 5, 500)}
    cv_obj = RandomizedSearchCV(
        rf_model,
        param_distributions=rf_params,
        n_iter=n_iter,
        cv=tscv,
        scoring="mean_absolute_percentage_error",
        random_state=random_state,
        verbose=0,
        n_jobs=-1,
    )
    cv_obj.fit(X_train, y_train)
    model = cv_obj.best_estimator_
else:
    model = Lasso(lasso.regressor.get_params(alpha))

In [102]:
model= lasso.regressor
model

Lasso(alpha=1.821885343465165e-05, random_state=123)

In [103]:
#we use the BlockBootstrap sampling method instead of the traditional bootstrap strategy for training the 
#model since the former is more suited for time series data. 
#Generate a sampling method, that block bootstraps the training set. 
#It can replace KFold, LeaveOneOut or SubSample as cv argument in the MapieRegressor class.
cv_mapiets = BlockBootstrap(n_resamplings=10, n_blocks=10, overlapping=False, random_state=123)
#Prediction intervals with out-of-fold residuals for time series: 
#This class implements the EnbPI strategy for estimating prediction intervals on single-output time series. 
# EnbPI method
mapie_enbpi = MapieTimeSeriesRegressor(model, method="enbpi", cv=cv_mapiets, agg_function="mean", n_jobs=-1)

In [104]:
print("EnbPI, with no partial_fit, width optimization")
mapie_enbpi = mapie_enbpi.fit(X_train, y_train)
mapie_enbpi

EnbPI, with no partial_fit, width optimization


MapieTimeSeriesRegressor(cv=BlockBootstrap(length=None, n_blocks=10, n_resamplings=10, overlapping=False,
        random_state=123),
                         estimator=Lasso(alpha=1.821885343465165e-05,
                                         random_state=123),
                         n_jobs=-1)

In [105]:
# Correspond to ‘Conformal prediction for dynamic time-series’: with a regular 
#.fit and .predict process, limiting the use of trainining set residuals to build prediction intervals
y_pred_npfit, y_pis_npfit = mapie_enbpi.predict(X_test, alpha=alpha, ensemble=True, optimize_beta=True)

In [106]:
y_pred_npfit

array([199332.35958213, 223281.88623987, 225907.64027748, 226192.24423415,
       227018.45540337, 227297.76996736, 217914.30916057])

In [107]:
y_pis_npfit  #[:, 0, :]: Lower bound of the prediction interval. [:, 1, :]: Upper bound of the prediction interval

array([[[178922.29772767],
        [211491.92933198]],

       [[202871.82438542],
        [235441.45598973]],

       [[205497.57842303],
        [238067.21002734]],

       [[205782.18237969],
        [238351.81398401]],

       [[206608.39354892],
        [239178.02515323]],

       [[206887.70811291],
        [239457.33971722]],

       [[197504.24730612],
        [230073.87891043]]])

In [108]:
#y_pred_lower_ips
y_pis_npfit[:, 0, 0] 

array([178922.29772767, 202871.82438542, 205497.57842303, 205782.18237969,
       206608.39354892, 206887.70811291, 197504.24730612])

In [109]:
#y_pred_upper_ips
y_pis_npfit[:, 1, 0]

array([211491.92933198, 235441.45598973, 238067.21002734, 238351.81398401,
       239178.02515323, 239457.33971722, 230073.87891043])

In [110]:
df_interval_nfit = pd.DataFrame({'y_test': y_test, 'y_preds': y_pred_npfit, 'lower':y_pis_npfit[:, 0, 0] , 'upper':y_pis_npfit[:, 1, 0]})
df_interval_nfit

,y_test,y_preds,lower,upper
FECHA,,,,
2023-09-24,203949.278,199332.360,178922.298,211491.929
2023-09-25,233453.085,223281.886,202871.824,235441.456
2023-09-26,236506.302,225907.640,205497.578,238067.210
2023-09-27,238964.909,226192.244,205782.182,238351.814
2023-09-28,235940.357,227018.455,206608.394,239178.025
2023-09-29,235929.693,227297.770,206887.708,239457.340
2023-09-30,228839.628,217914.309,197504.247,230073.879


In [111]:
import plotly.graph_objects as go
import pandas as pd
# Crear un DataFrame df con las columnas necesarias
df = df_interval_nfit[['y_test', 'y_preds', 'lower', 'upper']].copy()

# Crear la figura
fig = go.Figure()

# Agregar las series de datos con colores específicos
fig.add_trace(go.Scatter(x=df.index, y=df['y_test'], mode='lines', name='Real Data', line=dict(color='#bc8f8f')))
fig.add_trace(go.Scatter(x=df.index, y=df['y_preds'], mode='lines', name='Predictions', line=dict(color='#276981')))
fig.add_trace(go.Scatter(x=df.index, y=df['lower'], fill='tonexty', mode='lines', line_color='#437070', fillcolor='rgba(185, 203, 211, 0.8)', showlegend=False))
fig.add_trace(go.Scatter(x=df.index, y=df['upper'], fill='tonexty', mode='lines', line_color='#437070', fillcolor='rgba(185, 203, 211, 0.8)', name='Prediction Interval'))

# Estilo y diseño del gráfico
fig.update_layout(
    #title='Energy Demand Forecast with Prediction Interval',
    yaxis_title='Demand Forecast with Prediction Interval',
    xaxis_title='Date-daily',
    showlegend=True
)

# Mostrar el gráfico
fig.show()


In [112]:
#Effective coverage score obtained by the prediction intervals: cobertura efectiva: 
#La cobertura efectiva se obtiene estimando la fracción de etiquetas verdaderas que se encuentran dentro
#de los intervalos de predicción.
coverage_npfit = regression_coverage_score(y_test, y_pis_npfit[:, 0, 0], y_pis_npfit[:, 1, 0])
print("Coverage No parcial EnbPI fit", coverage_npfit)

# #Effective mean width score obtained by the prediction intervals: Ancho medio efectivo
width_npfit = regression_mean_width_score(y_pis_npfit[:, 0, 0], y_pis_npfit[:, 1, 0])
print("Mean width score No parcial EnbPI fit", width_npfit)

Coverage No parcial EnbPI fit 0.8571428571428571
Mean width score No parcial EnbPI fit 32569.631604313414


In [113]:
y_pred_npfit.shape

(7,)

In [114]:
y_pis_npfit.shape

(7, 2, 1)

In [115]:
#using .partial_fit in addition to .fit and .predict 
#allowing MAPIE to use new residuals from the test points as new data are becoming available.......

print("EnbPI with partial_fit, width optimization")
mapie_enbpi = mapie_enbpi.fit(X_train, y_train)

print("Size predictions EnbPI fit",y_pred_npfit.shape)
y_pred_pfit = np.zeros(y_pred_npfit.shape)
y_pis_pfit = np.zeros(y_pis_npfit.shape)

EnbPI with partial_fit, width optimization
Size predictions EnbPI fit (7,)


In [116]:
X_test.iloc[:gap, :]

,Covid,Cambio_nivel,Tiempo,VV_10_MEDIA_D,VV_10_VECT_MEDIA_D,PT_10_TT_D,THSM_MEDIA_D,TIPO_DIA_1--ENE,TIPO_DIA_1--MAY,TIPO_DIA_2--ENE,TIPO_DIA_20--JUL,TIPO_DIA_24--DIC,TIPO_DIA_25--DIC,TIPO_DIA_31--DIC,TIPO_DIA_7--AGO,TIPO_DIA_8--DIC,TIPO_DIA_DOALF,TIPO_DIA_DOALFENE,TIPO_DIA_DOMINGO,TIPO_DIA_DOVDIC,TIPO_DIA_DOVENE,TIPO_DIA_DSS,TIPO_DIA_JSS,TIPO_DIA_JUEVES,TIPO_DIA_JUVDIC,TIPO_DIA_JUVENE,TIPO_DIA_LF,TIPO_DIA_LFENE,TIPO_DIA_LUNES,TIPO_DIA_LUVDIC,TIPO_DIA_LUVENE,TIPO_DIA_MADLF,TIPO_DIA_MARTES,TIPO_DIA_MAVDIC,TIPO_DIA_MAVENE,TIPO_DIA_MIERCOLES,TIPO_DIA_MIVDIC,TIPO_DIA_MIVENE,TIPO_DIA_MSS,TIPO_DIA_SAALF,TIPO_DIA_SAALFENE,TIPO_DIA_SABADO,TIPO_DIA_SAVDIC,TIPO_DIA_SAVENE,TIPO_DIA_SSS,TIPO_DIA_VIERNES,TIPO_DIA_VIVDIC,TIPO_DIA_VIVENE,TIPO_DIA_VSS
FECHA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-09-24,0,1,2124,49.364,49.926,5.819,20.972,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [117]:

print("Zeros Arrays for y_pred and y_pis------------------------------------------------------------------------------------------ ")
y_pred_pfit[:gap], y_pis_pfit[:gap, :, :] = mapie_enbpi.predict(X_test.iloc[:gap, :], alpha=alpha, ensemble=True, optimize_beta=True)
print("gap: the update of the residuals and the one-step ahead predictions are performed sequentially in a loop--------------------")
print("Update the conformity_scores_ attribute when new data with known labels are available---------------------------------------")

Zeros Arrays for y_pred and y_pis------------------------------------------------------------------------------------------ 
gap: the update of the residuals and the one-step ahead predictions are performed sequentially in a loop--------------------
Update the conformity_scores_ attribute when new data with known labels are available---------------------------------------


In [118]:
#partial_fit: Update the conformity_scores_ attribute when new data with known labels are available
for step in range(gap, len(X_test), gap):
    mapie_enbpi.partial_fit(X_test.iloc[(step - gap):step, :],y_test.iloc[(step - gap):step], )
    (y_pred_pfit[step:step + gap], y_pis_pfit[step:step + gap, :, :], ) = mapie_enbpi.predict(
                                                                                        X_test.iloc[step:(step + gap), :],
                                                                                        alpha=alpha,
                                                                                        ensemble=True,
                                                                                        optimize_beta=True
    )

In [119]:
    
coverage_pfit = regression_coverage_score(
    y_test, y_pis_pfit[:, 0, 0], y_pis_pfit[:, 1, 0]
)
width_pfit = regression_mean_width_score(
    y_pis_pfit[:, 0, 0], y_pis_pfit[:, 1, 0]
)


print("y_pred_pfit--------------------------------",y_pred_pfit)
print("y_pis_pfit--------------------------------",y_pis_pfit)
print("Coverage  parcial EnbPI fit",coverage_pfit)
print("Mean width score parcial EnbPI fit",width_pfit)

y_pred_pfit-------------------------------- [199332.35958213 223281.88623987 225907.64027748 226192.24423415
 227018.45540337 227297.76996736 217914.30916057]
y_pis_pfit-------------------------------- [[[178922.29772767]
  [211491.92933198]]

 [[202871.82438542]
  [235441.45598973]]

 [[205497.57842303]
  [238067.21002734]]

 [[205782.18237969]
  [238351.81398401]]

 [[206608.39354892]
  [239253.34976296]]

 [[206887.70811291]
  [239532.66432694]]

 [[197504.24730612]
  [230149.20352015]]]
Coverage  parcial EnbPI fit 0.8571428571428571
Mean width score parcial EnbPI fit 32601.913579908723


In [120]:
y_pred_npfit

array([199332.35958213, 223281.88623987, 225907.64027748, 226192.24423415,
       227018.45540337, 227297.76996736, 217914.30916057])

In [121]:
y_pred_pfit

array([199332.35958213, 223281.88623987, 225907.64027748, 226192.24423415,
       227018.45540337, 227297.76996736, 217914.30916057])

In [122]:
y_pis_pfit[:, 0, 0] 

array([178922.29772767, 202871.82438542, 205497.57842303, 205782.18237969,
       206608.39354892, 206887.70811291, 197504.24730612])

In [123]:
y_pis_pfit[:, 1, 0]

array([211491.92933198, 235441.45598973, 238067.21002734, 238351.81398401,
       239253.34976296, 239532.66432694, 230149.20352015])

In [124]:
y_preds = [y_pred_npfit, y_pred_pfit]
y_pis = [y_pis_npfit, y_pis_pfit]
coverages = [coverage_npfit, coverage_pfit]
widths = [width_npfit, width_pfit]

In [125]:
y_pis

[array([[[178922.29772767],
         [211491.92933198]],
 
        [[202871.82438542],
         [235441.45598973]],
 
        [[205497.57842303],
         [238067.21002734]],
 
        [[205782.18237969],
         [238351.81398401]],
 
        [[206608.39354892],
         [239178.02515323]],
 
        [[206887.70811291],
         [239457.33971722]],
 
        [[197504.24730612],
         [230073.87891043]]]),
 array([[[178922.29772767],
         [211491.92933198]],
 
        [[202871.82438542],
         [235441.45598973]],
 
        [[205497.57842303],
         [238067.21002734]],
 
        [[205782.18237969],
         [238351.81398401]],
 
        [[206608.39354892],
         [239253.34976296]],
 
        [[206887.70811291],
         [239532.66432694]],
 
        [[197504.24730612],
         [230149.20352015]]])]

In [126]:
mapie_enbpi.conformity_score_function_

In [127]:
mapie_enbpi.score

<bound method RegressorMixin.score of MapieTimeSeriesRegressor(cv=BlockBootstrap(length=None, n_blocks=10, n_resamplings=10, overlapping=False,
        random_state=123),
                         estimator=Lasso(alpha=1.821885343465165e-05,
                                         random_state=123),
                         n_jobs=-1)>

In [128]:
mapie_enbpi.higher_quantiles_

array([12234.89435958])

In [129]:
mapie_enbpi.lower_quantiles_

array([-20410.06185445])

In [130]:
mapie_enbpi.conformity_scores_

array([ 2236.9382311 ,  3067.67054231, -3307.72346293, ...,
       12772.66494585,  8921.90116663,  8631.92283264])

In [131]:
color_deseado = '#276981'

# Crear el gráfico de dispersión con color personalizado
fig = go.Figure(data=go.Scatter(y=mapie_enbpi.conformity_scores_, mode='markers', marker=dict(color=color_deseado, size=4)))

# Configurar el diseño del gráfico
fig.update_layout(title='CP-scores',
                  xaxis_title='Index',
                  yaxis_title='Conformity scores')

# Mostrar el gráfico
fig.show()

In [132]:
from sklearn.metrics import mean_pinball_loss

mean_pinball_loss(df_interval_nfit['y_test'], df_interval_nfit['y_preds'])

4759.899054647521

In [133]:
plot_forecast(y_train, y_test, y_preds, y_pis, coverages, widths)

### Alternative with change point

In [140]:
print("EnbPI with partial_fit, width optimization")
mapie_enbpi = mapie_enbpi.fit(X_train, y_train)

y_pred_pfit = np.zeros(y_pred_npfit.shape)
y_pis_pfit = np.zeros(y_pis_npfit.shape)
conformity_scores_pfit, lower_quantiles_pfit, higher_quantiles_pfit = [], [], []
y_pred_pfit[:gap], y_pis_pfit[:gap, :, :] = mapie_enbpi.predict(
    X_test.iloc[:gap, :], alpha=alpha, ensemble=True, optimize_beta=True
)
for step in range(gap, len(X_test), gap):
    mapie_enbpi.partial_fit(
        X_test.iloc[(step - gap):step, :],
        y_test.iloc[(step - gap):step],
    )
    (
        y_pred_pfit[step:step + gap],
        y_pis_pfit[step:step + gap, :, :],
    ) = mapie_enbpi.predict(
        X_test.iloc[step:(step + gap), :],
        alpha=alpha,
        ensemble=True, 
        optimize_beta=True
    )
    conformity_scores_pfit.append(mapie_enbpi.conformity_scores_)
    lower_quantiles_pfit.append(mapie_enbpi.lower_quantiles_)
    higher_quantiles_pfit.append(mapie_enbpi.higher_quantiles_)
coverage_pfit = regression_coverage_score(
    y_test, y_pis_pfit[:, 0, 0], y_pis_pfit[:, 1, 0]
)
width_pfit = regression_mean_width_score(
    y_pis_pfit[:, 0, 0], y_pis_pfit[:, 1, 0]
)

EnbPI with partial_fit, width optimization


In [141]:
mapie_enbpi.conformity_scores_

array([ 2236.9382311 ,  3067.67054231, -3307.72346293, ...,
       12772.66494585,  8921.90116663,  8631.92283264])

In [142]:
y_preds = [y_pred_npfit, y_pred_pfit]
y_pis = [y_pis_npfit, y_pis_pfit]
coverages = [coverage_npfit, coverage_pfit]
widths = [width_npfit, width_pfit]

In [143]:
plot_forecast(y_train, y_test, y_preds, y_pis, coverages, widths)

In [161]:
window = 1
rolling_coverage_pfit, rolling_coverage_npfit = [], []
for i in range(window, len(y_test), 1):
    rolling_coverage_pfit.append(
        regression_coverage_score(
            y_test[i-window:i], y_pis_pfit[i-window:i, 0, 0], y_pis_pfit[i-window:i, 1, 0]
        )
    )
    rolling_coverage_npfit.append(
        regression_coverage_score(
            y_test[i-window:i], y_pis_npfit[i-window:i, 0, 0], y_pis_npfit[i-window:i, 1, 0]
        )
    )

In [162]:
rolling_coverage_npfit

[1.0, 1.0, 1.0, 0.0, 1.0, 1.0]

In [163]:
import plotly.graph_objects as go

def plot_rolling_coverage(y_test, rolling_coverage_npfit, rolling_coverage_pfit, window):
    fig = go.Figure()

    fig.update_layout(
        title="Rolling Coverage",
        xaxis_title="Time",
        yaxis_title=f"Rolling coverage [{window} hours]",
    )

    fig.add_trace(go.Scatter(
        x=y_test[window:].index,
        y=rolling_coverage_npfit,
        mode='lines',
        name="Without update of residuals"
    ))

    fig.add_trace(go.Scatter(
        x=y_test[window:].index,
        y=rolling_coverage_pfit,
        mode='lines',
        name="With update of residuals"
    ))

    fig.show()


In [164]:
higher_quantiles_pfit

[array([12159.56974986]),
 array([12159.56974986]),
 array([12159.56974986]),
 array([12234.89435958]),
 array([12234.89435958]),
 array([12234.89435958])]

In [165]:
import plotly.graph_objects as go

def plot_conformity_scores(conformity_scores_pfit, lower_quantiles_pfit, higher_quantiles_pfit):
    fig = go.Figure()

    for i, j in enumerate([0, -1]):
        fig.add_trace(go.Histogram(
            x=conformity_scores_pfit[j],
            histnorm='probability density',
            xbins=dict(
                start=-2.5,
                end=0.5,
                size=0.1
            ),
            marker=dict(color=f"rgba(0, 0, 256, 0.3)"),
            name=f"Conformity scores (step={j})"
        ))

        fig.add_shape(
            type="line",
            x0=lower_quantiles_pfit[j],
            y0=0,
            x1=lower_quantiles_pfit[j],
            y1=0.5,
            line=dict(
                dash='dash',
                color='rgba(0, 256, 0, 0.8)'
            )
        )

        fig.add_shape(
            type="line",
            x0=higher_quantiles_pfit[j],
            y0=0,
            x1=higher_quantiles_pfit[j],
            y1=0.5,
            line=dict(
                dash='dash',
                color='rgba(0, 256, 0, 0.8)'
            )
        )

    fig.update_layout(
        title="Conformity Scores",
        xaxis_title="Conformity Score",
        yaxis_title="Probability Density",
        legend=dict(
            x=1,
            y=0
        )
    )

    fig.show()


In [166]:
conformity_scores_pfit

[array([ 1865.28373685,  2571.37501904,  3628.26937921, ...,
        12953.05958209,  6476.77827968,  4616.91847787]),
 array([ 2571.37501904,  3628.26937921,  4954.72948665, ...,
         6476.77827968,  4616.91847787, 10171.19870013]),
 array([ 3628.26937921,  4954.72948665,  4869.84891756, ...,
         4616.91847787, 10171.19870013, 10598.66195252]),
 array([ 4954.72948665,  4869.84891756,  2236.9382311 , ...,
        10171.19870013, 10598.66195252, 12772.66494585]),
 array([ 4869.84891756,  2236.9382311 ,  3067.67054231, ...,
        10598.66195252, 12772.66494585,  8921.90116663]),
 array([ 2236.9382311 ,  3067.67054231, -3307.72346293, ...,
        12772.66494585,  8921.90116663,  8631.92283264])]

In [167]:
lower_quantiles_pfit

[array([-20410.06185445]),
 array([-20410.06185445]),
 array([-20410.06185445]),
 array([-20410.06185445]),
 array([-20410.06185445]),
 array([-20410.06185445])]

In [168]:
higher_quantiles_pfit

[array([12159.56974986]),
 array([12159.56974986]),
 array([12159.56974986]),
 array([12234.89435958]),
 array([12234.89435958]),
 array([12234.89435958])]

In [169]:
plot_conformity_scores(conformity_scores_pfit, lower_quantiles_pfit, higher_quantiles_pfit)

## Wrapper keras with scikeras

In [3]:
#Import Data..............

df=pd.read_parquet('df.parquet')
columnas_indeseadas =['HR_CAL_MEDIA_D', 'VVAG_MEDIA_D'] #columna con correlación 0 con la demanda
columnas_indeseadas += [f'Demanda_SIN_{i}' for i in range(1, 29)]
# columnas_indeseadas += [f'Demanda_SIN_{i}' for i in range(7, 29, 7)]
print("columnas rezagadas que no son necesarias porque skforecast rezaga internamente",columnas_indeseadas)
df = df.loc[:, ~df.columns.isin(columnas_indeseadas)]
print(df.head(1))
print("--------------------------------------------------------------------------------------------------")
print(df.tail(1))

columnas rezagadas que no son necesarias porque skforecast rezaga internamente ['HR_CAL_MEDIA_D', 'VVAG_MEDIA_D', 'Demanda_SIN_1', 'Demanda_SIN_2', 'Demanda_SIN_3', 'Demanda_SIN_4', 'Demanda_SIN_5', 'Demanda_SIN_6', 'Demanda_SIN_7', 'Demanda_SIN_8', 'Demanda_SIN_9', 'Demanda_SIN_10', 'Demanda_SIN_11', 'Demanda_SIN_12', 'Demanda_SIN_13', 'Demanda_SIN_14', 'Demanda_SIN_15', 'Demanda_SIN_16', 'Demanda_SIN_17', 'Demanda_SIN_18', 'Demanda_SIN_19', 'Demanda_SIN_20', 'Demanda_SIN_21', 'Demanda_SIN_22', 'Demanda_SIN_23', 'Demanda_SIN_24', 'Demanda_SIN_25', 'Demanda_SIN_26', 'Demanda_SIN_27', 'Demanda_SIN_28']
       FECHA TIPO_DIA  Demanda_SIN  Covid  Cambio_nivel  Tiempo  precipitacion  Temperatura  HRA2_MN_D  HRA2_MX_D  THSM_MEDIA_D  THSM_MX_D  VV_10_MEDIA_D  VV_10_VECT_MEDIA_D  THSM_MN_D  PT_2_TT_D  PT_2_MX_D  PT_10_MX_D  PT_10_TT_D  TSSM_MEDIA_D  TSSM_MN_D  TSSM_MX_D  PTPG_TT_D  BSHG_TT_D  TPR_CAL
0 2017-12-01  VIERNES   193865.750      0             0       1          9.589       22.203  

In [4]:
#Correlación variables y filtrado........
#['THSM_MEDIA_D','PTPG_TT_D']
correlacion_con_demanda = df.corr()['Demanda_SIN']
nombres_variables = correlacion_con_demanda[(correlacion_con_demanda >= 0.2) | (correlacion_con_demanda <= -0.2)]
print(nombres_variables)
nombres_variables=nombres_variables.index.tolist()
nombres_variables +=['THSM_MEDIA_D'] #'PTPG_TT_D' #'VVAG_MEDIA_D'
nombres_variables.remove('Demanda_SIN')
df=df[['FECHA', 'TIPO_DIA', 'Demanda_SIN'] + nombres_variables]
print("columnas finales", df.columns)
df.head(2)

Demanda_SIN           1.000
Covid                -0.246
Cambio_nivel          0.352
Tiempo                0.595
VV_10_MEDIA_D         0.213
VV_10_VECT_MEDIA_D    0.210
PT_10_TT_D           -0.291
Name: Demanda_SIN, dtype: float64
columnas finales Index(['FECHA', 'TIPO_DIA', 'Demanda_SIN', 'Covid', 'Cambio_nivel', 'Tiempo', 'VV_10_MEDIA_D', 'VV_10_VECT_MEDIA_D', 'PT_10_TT_D', 'THSM_MEDIA_D'], dtype='object')


,FECHA,TIPO_DIA,Demanda_SIN,Covid,Cambio_nivel,Tiempo,VV_10_MEDIA_D,VV_10_VECT_MEDIA_D,PT_10_TT_D,THSM_MEDIA_D
0,2017-12-01,VIERNES,193865.750,0,0,1,1.703,0.935,6.232,19.113
1,2017-12-02,SABADO,184393.750,0,0,2,1.757,0.930,7.888,19.166


In [5]:
#Imputation: Rolling mean........................

columns_rolling = ['VV_10_MEDIA_D', 'VV_10_VECT_MEDIA_D','PT_10_TT_D' ,'THSM_MEDIA_D']
window_size=60
df = tool.exploratory_tools.fill_null_values_with_rolling_mean(df, columns_rolling, window_size)

In [6]:
# Transformer one hot encoding and formatting-.......

print("--------------------------------------------------------------------------------------------------------------------------------------")
# ==============================================================================
one_hot_encoder = make_column_transformer(
                      #(StandardScaler(), ['Demanda_SIN']),
                      (   
                          OneHotEncoder(sparse_output=False, drop='if_binary'),
                          ['TIPO_DIA'],
                      ),
                      remainder="passthrough",
                      verbose_feature_names_out=False,
                  ).set_output(transform="pandas")

df = one_hot_encoder.fit_transform(df)

#-------------------------------------------------------------------------------------------------------------------------------------------
#Variable TIPO_DIA
#-------------------------------------------------------------------------------------------------------------------------------------------
columnas_tipo_dia = [col for col in df.columns if col.startswith('TIPO_DIA_')]
variables_exogenas = ['FECHA'] + nombres_variables + columnas_tipo_dia
variable_dependiente = ['Demanda_SIN']
df = df[variable_dependiente + variables_exogenas] 

#-------------------------------------------------------------------------------------------------------------------------------------------
# Codificación de Fecha...
#-------------------------------------------------------------------------------------------------------------------------------------------
df['FECHA'] = pd.to_datetime(df['FECHA'], format='%Y-%m-%d')
df = df.set_index('FECHA')
df = df.asfreq('D')


df.head(2)

--------------------------------------------------------------------------------------------------------------------------------------


,Demanda_SIN,Covid,Cambio_nivel,Tiempo,VV_10_MEDIA_D,VV_10_VECT_MEDIA_D,PT_10_TT_D,THSM_MEDIA_D,TIPO_DIA_1--ENE,TIPO_DIA_1--MAY,TIPO_DIA_2--ENE,TIPO_DIA_20--JUL,TIPO_DIA_24--DIC,TIPO_DIA_25--DIC,TIPO_DIA_31--DIC,TIPO_DIA_7--AGO,TIPO_DIA_8--DIC,TIPO_DIA_DOALF,TIPO_DIA_DOALFENE,TIPO_DIA_DOMINGO,TIPO_DIA_DOVDIC,TIPO_DIA_DOVENE,TIPO_DIA_DSS,TIPO_DIA_JSS,TIPO_DIA_JUEVES,TIPO_DIA_JUVDIC,TIPO_DIA_JUVENE,TIPO_DIA_LF,TIPO_DIA_LFENE,TIPO_DIA_LUNES,TIPO_DIA_LUVDIC,TIPO_DIA_LUVENE,TIPO_DIA_MADLF,TIPO_DIA_MARTES,TIPO_DIA_MAVDIC,TIPO_DIA_MAVENE,TIPO_DIA_MIERCOLES,TIPO_DIA_MIVDIC,TIPO_DIA_MIVENE,TIPO_DIA_MSS,TIPO_DIA_SAALF,TIPO_DIA_SAALFENE,TIPO_DIA_SABADO,TIPO_DIA_SAVDIC,TIPO_DIA_SAVENE,TIPO_DIA_SSS,TIPO_DIA_VIERNES,TIPO_DIA_VIVDIC,TIPO_DIA_VIVENE,TIPO_DIA_VSS
FECHA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-12-01,193865.750,0,0,1,1.703,0.935,6.232,19.113,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000
2017-12-02,184393.750,0,0,2,1.757,0.930,7.888,19.166,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [7]:
# Separación datos train-val-test................
#-------------------------------------------------------------------------------------------------------------------------------------------

datos = df[df.index >= '2017-12-01']
fin_train = '2023-05-31'
fin_validacion = '2023-09-23'

datos_train = datos[datos.index <= fin_train].copy()
datos_val = datos[(datos.index > fin_train) & (datos.index <= fin_validacion)].copy()
datos_test = datos[datos.index > fin_validacion].copy()

print(f"Fechas train      : {datos_train.index.min()} --- {datos_train.index.max()}  (n={len(datos_train)})")
print(f"Fechas validacion : {datos_val.index.min()} --- {datos_val.index.max()}  (n={len(datos_val)})")
print(f"Fechas test       : {datos_test.index.min()} --- {datos_test.index.max()}  (n={len(datos_test)})")
print("-------------------------------------------------------------------------------------------------------------------------------------")

datos_test.head(2)

Fechas train      : 2017-12-01 00:00:00 --- 2023-05-31 00:00:00  (n=2008)
Fechas validacion : 2023-06-01 00:00:00 --- 2023-09-23 00:00:00  (n=115)
Fechas test       : 2023-09-24 00:00:00 --- 2023-09-30 00:00:00  (n=7)
-------------------------------------------------------------------------------------------------------------------------------------


,Demanda_SIN,Covid,Cambio_nivel,Tiempo,VV_10_MEDIA_D,VV_10_VECT_MEDIA_D,PT_10_TT_D,THSM_MEDIA_D,TIPO_DIA_1--ENE,TIPO_DIA_1--MAY,TIPO_DIA_2--ENE,TIPO_DIA_20--JUL,TIPO_DIA_24--DIC,TIPO_DIA_25--DIC,TIPO_DIA_31--DIC,TIPO_DIA_7--AGO,TIPO_DIA_8--DIC,TIPO_DIA_DOALF,TIPO_DIA_DOALFENE,TIPO_DIA_DOMINGO,TIPO_DIA_DOVDIC,TIPO_DIA_DOVENE,TIPO_DIA_DSS,TIPO_DIA_JSS,TIPO_DIA_JUEVES,TIPO_DIA_JUVDIC,TIPO_DIA_JUVENE,TIPO_DIA_LF,TIPO_DIA_LFENE,TIPO_DIA_LUNES,TIPO_DIA_LUVDIC,TIPO_DIA_LUVENE,TIPO_DIA_MADLF,TIPO_DIA_MARTES,TIPO_DIA_MAVDIC,TIPO_DIA_MAVENE,TIPO_DIA_MIERCOLES,TIPO_DIA_MIVDIC,TIPO_DIA_MIVENE,TIPO_DIA_MSS,TIPO_DIA_SAALF,TIPO_DIA_SAALFENE,TIPO_DIA_SABADO,TIPO_DIA_SAVDIC,TIPO_DIA_SAVENE,TIPO_DIA_SSS,TIPO_DIA_VIERNES,TIPO_DIA_VIVDIC,TIPO_DIA_VIVENE,TIPO_DIA_VSS
FECHA,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2023-09-24,203949.278,0,1,2124,49.364,49.926,5.819,20.972,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
2023-09-25,233453.085,0,1,2125,49.389,49.956,5.880,20.969,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [45]:
#Split and reshape.....

#DATOS DE ENTRENAMIENTO
X_train = datos_train.drop("Demanda_SIN", axis=1)
y_train = datos_train["Demanda_SIN"]
X_train = X_train.values
X_train = X_train.reshape((X_train.shape[0],1,  X_train.shape[1]))
print(X_train.shape)

# DATOS DE VALIDACIÓN
X_val = datos_val.drop("Demanda_SIN", axis=1)
y_val = datos_val["Demanda_SIN"]
X_val=X_val.values
X_val = X_val.reshape((X_val.shape[0],1,  X_val.shape[1]))
print(X_val.shape)

# DATOS DE TEST
X_test = datos_test.drop("Demanda_SIN", axis=1)
y_test = datos_test["Demanda_SIN"]
X_test=X_test.values
X_test = X_test.reshape((X_test.shape[0],1,  X_test.shape[1]))
print(X_test.shape)

(2008, 1, 49)
(115, 1, 49)
(7, 1, 49)


In [9]:
early_stopping = EarlyStopping(monitor="val_loss", patience=20)
early_stopping

In [10]:
from scikeras.wrappers import KerasRegressor
from src.Utilities import utilities as tool
from skforecast.model_selection import backtesting_forecaster
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import grid_search_forecaster

In [42]:
def create_simple_rnn():
    model = Sequential() 
    model.add(SimpleRNN(96)) 
    model.add(Dropout(1))
    model.add(SimpleRNN(128))
    model.add(SimpleRNN(32))
    model.add(SimpleRNN(96))
    model.add(Dropout(1))
    model.add(Dense(1)) 
    model.compile(optimizer="adam", loss="mean_squared_error")
    return model

In [84]:
def create_simple_rnn_model(rnn_units, dropout_rate, n_layers, final_units, final_activation):
    model = Sequential()
    model.add(SimpleRNN(units=rnn_units, activation='relu', return_sequences=True, input_shape=(1, 49)))
    model.add(Dropout(rate=dropout_rate))

    for i in range(n_layers):
        model.add(SimpleRNN(units=rnn_units, activation='relu', return_sequences=True))

    model.add(SimpleRNN(units=final_units, activation=final_activation))
    model.add(Dropout(rate=dropout_rate))

    model.add(Dense(units=1, activation='linear'))  # Ensure the last layer is suitable for regression
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mse'])

    return model

In [85]:
from scikeras.wrappers import KerasRegressor

keras_regressor_model=KerasRegressor(model=create_simple_rnn,build_fn=create_simple_rnn ,  batch_size=3, loss = 'mean_squared_error', random_state=123, validation_batch_size= 7, callbacks=early_stopping, metrics=['mean_squared_error'] )

In [65]:
type(keras_regressor_model)

scikeras.wrappers.KerasRegressor

In [83]:
forecaster.regressor

KerasRegressor(
	model=<function create_simple_rnn at 0x0000027F0E832B80>
	build_fn=<function create_simple_rnn at 0x0000027F0E832B80>
	warm_start=False
	random_state=123
	optimizer=rmsprop
	loss=mean_squared_error
	metrics=['mean_squared_error']
	batch_size=3
	validation_batch_size=7
	verbose=1
	callbacks=<keras.src.callbacks.EarlyStopping object at 0x0000027F0C688370>
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
)

In [86]:
# Sin búsqueda de parámetros.........

# ForecasterAutoreg: keras_regressor_model
# ==============================================================================
forecaster = ForecasterAutoreg(
                 regressor     = keras_regressor_model,
                 lags          = [1, 2, 3, 4, 5, 6, 7, 14,21, 28]
             )


columnas_tipo_dia = [col for col in df.columns if col.startswith('TIPO_DIA_')]
variables_exogenas = nombres_variables + columnas_tipo_dia

forecaster.fit(y=df.loc[:fin_validacion, 'Demanda_SIN'], exog=df.loc[:fin_validacion, variables_exogenas])

#Sin Búsqueda de hiperparametros
# Backtesting datos de test
# ==============================================================================
metric, predicciones = backtesting_forecaster(
                            forecaster         = forecaster,
                            y                    = datos.loc[:fin_validacion, 'Demanda_SIN'],
                            exog                 = datos.loc[:fin_validacion, variables_exogenas],
                            initial_train_size = len(datos.loc[:fin_train]),
                            fixed_train_size   = False, #Incremental
                            allow_incomplete_fold =True,
                            steps              = 7, #len(datos.loc[fin_train:fin_validacion])
                            metric             = ['mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error', 'mean_squared_log_error'],
                            refit              = True, #7
                            interval           = [2.5, 97.5],
                            random_state	  = 123,
                            n_jobs             = 'auto',
                            verbose            = True,
                            show_progress      = True,
                       )
print(metric)
print(f"Error de backtest_mean_squared_error: {metric[0]:.2f}")
print(f"Error de backtest_mean_absolute_error: {metric[1]:.2f}")
print(f"Error de backtest_mean_absolute_percentage_error: {metric[2]:.2f}")
print(f"Error de backtest_mean_squared_log_error: {metric[3]:.2f}")

predicciones
y_true = datos.loc[predicciones.index, 'Demanda_SIN']
mae = mean_absolute_error(y_true, predicciones['pred'])
mape = (mae / y_true).mean() * 100
print("MAPE:", mape)


# Importancia predictores
# ==============================================================================
print(forecaster.get_feature_importances())


#Grafico de predicciones vs real........

fig = go.Figure()
fig.add_trace(go.Scatter(x=datos.loc[predicciones.index, 'Demanda_SIN'].index,
                         y=datos.loc[predicciones.index, 'Demanda_SIN'],
                         mode='lines',
                         name='test',
                         line=dict(color='#bc8f8f'))) 
fig.add_trace(go.Scatter(x=predicciones.index,
                         y=predicciones['pred'],
                         mode='lines',
                         name='predicción',
                         line=dict(color='#276981'))) 
fig.update_layout(title='Predicción vs demanda real')
fig.update_layout(legend=dict(title='Series', orientation='h', y=1.02, xanchor='right', x=1))
fig.show()

TypeError: object of type 'NoneType' has no len()

In [15]:
# Models to compare Lasso.....
import random
random.seed(123)

models = [
          keras_regressor_model
          ]

# Hyperparameter to search for each model
param_grids = {
                'keras_regressor_model': {
                'rnn_units': [32, 64, 96, 128],
                'activation': ["relu", "selu", "sigmoid", "tanh"],
                'dropout_rate': [0.2, 0.3, 0.4, 0.5, 0.6, 0.7],
                'n_layers': [1, 2, 3, 4],
                'final_units': [32, 64, 96, 128],
                'final_activation': ["relu", "selu", "sigmoid", "tanh"]
    }
}

                        
# Lags used as predictors
lags_grid = [6, 7 ,14, 21, 28]

columnas_tipo_dia = [col for col in df.columns if col.startswith('TIPO_DIA_')]
variables_exogenas = nombres_variables + columnas_tipo_dia

df_results = pd.DataFrame()
for i, model in enumerate(models):
    print(f"-------------------------")
    print(f"Grid search for regressor: {model}")
    print(f"-------------------------")

    forecaster = ForecasterAutoreg(
                     regressor = model,
                     lags      = [1, 2, 3, 4, 5, 6, 7, 14, 21, 28],
                     transformer_y = StandardScaler()
                 )

    # Regressor hyperparameters
    param_grid = param_grids[list(param_grids)[i]]

    results_grid = grid_search_forecaster(
                       forecaster         = forecaster,
                       y                  = datos.loc[:fin_validacion, 'Demanda_SIN'],
                       exog               = datos.loc[:fin_validacion, variables_exogenas],
                       param_grid         = param_grid,
                       lags_grid          = lags_grid,
                       steps              = len(datos.loc[fin_train:fin_validacion]),
                       refit              = True, #True
                       metric             = ['mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error', 'mean_squared_log_error'],
                       initial_train_size = len(datos.loc[:fin_train]),
                       fixed_train_size   = True, # False
                       allow_incomplete_fold =True,
                       return_best        = True, #True
                       n_jobs             = 'auto',
                       verbose            = False, #True
                       show_progress      = True
                   )
    
    # Create a column with model name
    results_grid['model'] = list(param_grids)[i]
    
    df_results = pd.concat([df_results, results_grid])

df_results = df_results.sort_values(by='mean_squared_error')



# Backtesting datos de test
# ==============================================================================
metric, predicciones = backtesting_forecaster(
                            forecaster         = forecaster,
                            y                    = datos.loc[:fin_validacion, 'Demanda_SIN'],
                            exog                 = datos.loc[:fin_validacion, variables_exogenas],
                            initial_train_size = len(datos.loc[:fin_train]),
                            fixed_train_size   = False, #Incremental
                            allow_incomplete_fold =True,
                            steps              = 7, #len(datos.loc[fin_train:fin_validacion])
                            metric             = ['mean_squared_error', 'mean_absolute_error', 'mean_absolute_percentage_error', 'mean_squared_log_error'],
                            refit              = True, #7
                            interval           = [2.5, 97.5],
                            random_state	  = 123,
                            n_jobs             = 'auto',
                            verbose            = True,
                            show_progress      = True,
                       )
print(metric)
print(f"Error de backtest_mean_squared_error: {metric[0]:.2f}")
print(f"Error de backtest_mean_absolute_error: {metric[1]:.2f}")
print(f"Error de backtest_mean_absolute_percentage_error: {metric[2]:.2f}")
print(f"Error de backtest_mean_squared_log_error: {metric[3]:.2f}")

predicciones
y_true = datos.loc[predicciones.index, 'Demanda_SIN']
mae = mean_absolute_error(y_true, predicciones['pred'])
mape = (mae / y_true).mean() * 100
print("MAPE:", mape)


# Importancia predictores
# ==============================================================================
print(forecaster.get_feature_importances())

#Grafico de predicciones vs real........

fig = go.Figure()
fig.add_trace(go.Scatter(x=datos.loc[predicciones.index, 'Demanda_SIN'].index,
                         y=datos.loc[predicciones.index, 'Demanda_SIN'],
                         mode='lines',
                         name='test',
                         line=dict(color='#bc8f8f'))) 
fig.add_trace(go.Scatter(x=predicciones.index,
                         y=predicciones['pred'],
                         mode='lines',
                         name='predicción',
                         line=dict(color='#276981'))) 
fig.update_layout(title='Predicción vs demanda real')
fig.update_layout(legend=dict(title='Series', orientation='h', y=1.02, xanchor='right', x=1))
fig.show()

-------------------------
Grid search for regressor: KerasRegressor(
	model=None
	build_fn=<function create_simple_rnn at 0x00000235FF38B430>
	warm_start=False
	random_state=None
	optimizer=rmsprop
	loss=None
	metrics=None
	batch_size=3
	validation_batch_size=None
	verbose=1
	callbacks=None
	validation_split=0.0
	shuffle=True
	run_eagerly=False
	epochs=1
	nb_epoch=100
)
-------------------------
Number of models compared: 30720.


lags grid:   0%|          | 0/5 [00:00<?, ?it/s]

ValueError: Invalid parameter activation for estimator KerasRegressor.
This issue can likely be resolved by setting this parameter in the KerasRegressor constructor:
`KerasRegressor(activation=relu)`
Check the list of available parameters with `estimator.get_params().keys()`

In [29]:
keras_regressor_model.get_params().keys()

dict_keys(['model', 'build_fn', 'warm_start', 'random_state', 'optimizer', 'loss', 'metrics', 'batch_size', 'validation_batch_size', 'verbose', 'callbacks', 'validation_split', 'shuffle', 'run_eagerly', 'epochs'])

# Ejemplo

In [10]:
# url_file = "https://raw.githubusercontent.com/scikit-learn-contrib/MAPIE/master/examples/data/demand_temperature.csv"
# demand_df = pd.read_csv(
#     url_file, parse_dates=True, index_col=0
# )

# demand_df["Date"] = pd.to_datetime(demand_df.index)
# demand_df["Weekofyear"] = demand_df.Date.dt.isocalendar().week.astype("int64")
# demand_df["Weekday"] = demand_df.Date.dt.isocalendar().day.astype("int64")
# demand_df["Hour"] = demand_df.index.hour
# n_lags = 5
# for hour in range(1, n_lags):
#     demand_df[f"Lag_{hour}"] = demand_df["Demand"].shift(hour)

In [11]:
# demand_df.head()

,Demand,Temperature,Date,Weekofyear,Weekday,Hour,Lag_1,Lag_2,Lag_3,Lag_4
2014-01-01 00:00:00,3.794,18.050,2014-01-01 00:00:00,1,3,0,NaN,NaN,NaN,NaN
2014-01-01 01:00:00,3.418,17.200,2014-01-01 01:00:00,1,3,1,3.794,NaN,NaN,NaN
2014-01-01 02:00:00,3.152,16.450,2014-01-01 02:00:00,1,3,2,3.418,3.794,NaN,NaN
2014-01-01 03:00:00,3.026,16.650,2014-01-01 03:00:00,1,3,3,3.152,3.418,3.794,NaN
2014-01-01 04:00:00,3.022,16.400,2014-01-01 04:00:00,1,3,4,3.026,3.152,3.418,3.794


In [12]:
num_test_steps = 24 * 7
num_test_steps
print(demand_df.shape)


demand_train = demand_df.iloc[:-num_test_steps, :].copy()
print(demand_train.shape)


demand_test = demand_df.iloc[-num_test_steps:, :].copy()
print(demand_test.shape)



features = ["Weekofyear", "Weekday", "Hour", "Temperature"] 
features += [f"Lag_{hour}" for hour in range(1, n_lags)]
print(features)


168

In [49]:
# X_train = demand_train.loc[
#     ~np.any(demand_train[features].isnull(), axis=1), features
# ]
print(type(X_train))
print(len(X_train))

<class 'pandas.core.frame.DataFrame'>
2123


In [50]:
# y_train = demand_train.loc[X_train.index, "Demand"]
type(y_train)
print(len(y_train))

2123


In [51]:
# X_test = demand_test.loc[:, features]
print(type(X_test))
print(len(X_test))
# y_test = demand_test["Demand"]
print(type(y_test))
print(len(y_test))

<class 'pandas.core.frame.DataFrame'>
7
<class 'pandas.core.series.Series'>
7


In [52]:
# Crear el gráfico
trace1 = go.Scatter(x = list(range(len(y_train))), y = y_train, mode = 'lines', name = 'y_train')
trace2 = go.Scatter(x = list(range(len(y_train), len(y_train) + len(y_test))), y = y_test, mode = 'lines', name = 'y_test')
layout = go.Layout(title='Hourly demand (GW)', xaxis=dict(title='Time'), yaxis=dict(title='Hourly demand (GW)'))
fig = go.Figure(data=[trace1, trace2], layout=layout)

# Mostrar el gráfico
pyo.iplot(fig)

In [23]:
model_params_fit_not_done = False
if model_params_fit_not_done:
    # CV parameter search
    n_iter = 100
    n_splits = 5
    tscv = TimeSeriesSplit(n_splits=n_splits)
    random_state = 59
    rf_model = RandomForestRegressor(random_state=random_state)
    rf_params = {"max_depth": randint(2, 30), "n_estimators": randint(10, 100)}
    cv_obj = RandomizedSearchCV(
        rf_model,
        param_distributions=rf_params,
        n_iter=n_iter,
        cv=tscv,
        scoring="neg_root_mean_squared_error",
        random_state=random_state,
        verbose=0,
        n_jobs=-1,
    )
    cv_obj.fit(X_train, y_train)
    model = cv_obj.best_estimator_
else:
    # Model: Random Forest previously optimized with a cross-validation
    model = RandomForestRegressor(
        max_depth=10, n_estimators=50, random_state=59)

In [53]:
model

Lasso(alpha=1.821885343465165e-05, random_state=123)

In [54]:
alpha = 0.05
gap = 1
cv_mapiets = BlockBootstrap(
    n_resamplings=100, length=48, overlapping=True, random_state=59
)
print(type(cv_mapiets))

<class 'mapie.subsample.BlockBootstrap'>


['_BaseCrossValidator__metadata_request__split',
 '__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_build_request_for_signature',
 '_get_default_requests',
 '_get_metadata_request',
 '_iter_test_indices',
 '_iter_test_masks',
 'get_metadata_routing',
 'get_n_splits',
 'length',
 'n_blocks',
 'n_resamplings',
 'overlapping',
 'random_state',
 'split']

In [55]:
cv_mapiets

BlockBootstrap(length=48, n_blocks=None, n_resamplings=100, overlapping=True,
        random_state=59)

In [56]:
mapie_enbpi = MapieTimeSeriesRegressor(
    model, method="enbpi", cv=cv_mapiets, agg_function="mean", n_jobs=-1
)

print(type(mapie_enbpi))
mapie_enbpi

<class 'mapie.regression.time_series_regression.MapieTimeSeriesRegressor'>


MapieTimeSeriesRegressor(cv=BlockBootstrap(length=48, n_blocks=None, n_resamplings=100, overlapping=True,
        random_state=59),
                         estimator=Lasso(alpha=1.821885343465165e-05,
                                         random_state=123),
                         n_jobs=-1)

In [57]:
X_test.shape

(7, 49)

In [58]:
X_train.shape

(2123, 49)

In [59]:
print("EnbPI, with no partial_fit, width optimization")
mapie_enbpi = mapie_enbpi.fit(X_train, y_train)
y_pred_npfit, y_pis_npfit = mapie_enbpi.predict(
    X_test, alpha=alpha, ensemble=True, optimize_beta=True
)
y_pred_npfit

EnbPI, with no partial_fit, width optimization


array([200438.16120549, 224432.90790909, 227039.07974485, 227297.16163706,
       228110.70788665, 228366.82079458, 219105.07032422])

In [60]:
len(y_pred_npfit)

7

In [61]:
y_pis_npfit

array([[[190331.55822793],
        [208241.04076683]],

       [[214326.30493153],
        [232235.78747043]],

       [[216932.4767673 ],
        [234841.95930619]],

       [[217190.5586595 ],
        [235100.0411984 ]],

       [[218004.10490909],
        [235913.58744799]],

       [[218260.21781702],
        [236169.70035592]],

       [[208998.46734666],
        [226907.94988556]]])

In [62]:
len(y_pis_npfit)

7

In [63]:
coverage_npfit = regression_coverage_score(
    y_test, y_pis_npfit[:, 0, 0], y_pis_npfit[:, 1, 0]
)
width_npfit = regression_mean_width_score(
    y_pis_npfit[:, 0, 0], y_pis_npfit[:, 1, 0]
)

In [64]:
print("EnbPI with partial_fit, width optimization")
mapie_enbpi = mapie_enbpi.fit(X_train, y_train)

y_pred_pfit = np.zeros(y_pred_npfit.shape)
y_pis_pfit = np.zeros(y_pis_npfit.shape)
y_pred_pfit[:gap], y_pis_pfit[:gap, :, :] = mapie_enbpi.predict(
    X_test.iloc[:gap, :], alpha=alpha, ensemble=True, optimize_beta=True
)
for step in range(gap, len(X_test), gap):
    mapie_enbpi.partial_fit(
        X_test.iloc[(step - gap):step, :],
        y_test.iloc[(step - gap):step],
    )
    (
        y_pred_pfit[step:step + gap],
        y_pis_pfit[step:step + gap, :, :],
    ) = mapie_enbpi.predict(
        X_test.iloc[step:(step + gap), :],
        alpha=alpha,
        ensemble=True,
        optimize_beta=True
    )
coverage_pfit = regression_coverage_score(
    y_test, y_pis_pfit[:, 0, 0], y_pis_pfit[:, 1, 0]
)
width_pfit = regression_mean_width_score(
    y_pis_pfit[:, 0, 0], y_pis_pfit[:, 1, 0]
)

EnbPI with partial_fit, width optimization


In [65]:
y_preds = [y_pred_npfit, y_pred_pfit]
y_pis = [y_pis_npfit, y_pis_pfit]
coverages = [coverage_npfit, coverage_pfit]
widths = [width_npfit, width_pfit]


In [67]:
plot_forecast(y_train, y_test, y_preds, y_pis, coverages, widths)